# Nantes: The West Coast of France

## An analysis of the neighbourhoods in the primary city of Nantes (excluding suburbs) with special emphasis on schools and top residential areas.

Problem:  There is very little useful information in English for new or would-be immigrants to Nantes about the different neighbourhoods, especially pertaining to early education and desirable residential areas.

Objective: To help new or would-be immigrants, who are not yet fluent in French, to find suitable schools for their children and accomodation in Nantes.
This report will provide information on :  
(1) schools (elementary vs. high school, public vs private, french vs bilingual/international),  
(2) detailed information such as venues of each neighbourhood,  
(3) areas to avoid,  
(4) average property prices of each neighbourhood, and  
(5) top areas to live. 

In [2]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import geocoder
import folium

print('Libraries imported.')

Libraries imported.


In [3]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_quartiers_de_Nantes'

In [4]:
data  = requests.get(url).text 
soup = BS(data, 'html5lib')

In [5]:
table = soup.find('table')
for sup in table.find_all('sup'):
    sup.unwrap()
for span in table.find_all('span'):
    span.unwrap()
for a in table.find_all('a'):
    a.unwrap()

In [6]:
neighbourhoods = []
for row in table.find_all("tr"):
    if len(row.find_all('td')) >= 1:
        col = row.find_all('td')
        if (col != []):
            Neighbourhood = col[0].text
            Population = col[1].text
        neighbourhoods.append([Neighbourhood, Population])
neighbourhoods

[['Centre-ville[5]', '28 485'],
 ['Bellevue - Chantenay - Sainte-Anne[6]', '25 000'],
 ['Dervallières - Zola[7]', '35 000'],
 ['Hauts-Pavés - Saint-Félix[8]', '35 800'],
 ['Malakoff - Saint-Donatien[9]', '34 669'],
 ['Île de Nantes[10]', '15 818'],
 ['Breil - Barberie[11]', '24 418'],
 ['Nantes Nord[12]', '24 833'],
 ['Nantes Erdre[13]', '26 738'],
 ['Doulon - Bottière[14]', '30 147'],
 ['Nantes Sud[15]', '10 532']]

In [7]:
neighbourhoods1 = pd.DataFrame(neighbourhoods, columns = ['Neighbourhood', 'Population'])

In [8]:
neighbourhoods1

,Neighbourhood,Population
0,Centre-ville[5],28 485
1,Bellevue - Chantenay - Sainte-Anne[6],25 000
2,Dervallières - Zola[7],35 000
3,Hauts-Pavés - Saint-Félix[8],35 800
4,Malakoff - Saint-Donatien[9],34 669
5,Île de Nantes[10],15 818
6,Breil - Barberie[11],24 418
7,Nantes Nord[12],24 833
8,Nantes Erdre[13],26 738
9,Doulon - Bottière[14],30 147


In [9]:
import re
def purify(txt):
    return re.sub("\[.+\]", "", txt)

neighbourhoods1['Neighbourhood'] = neighbourhoods1['Neighbourhood'].apply(purify).copy()

In [10]:
neighbourhoods1

,Neighbourhood,Population
0,Centre-ville,28 485
1,Bellevue - Chantenay - Sainte-Anne,25 000
2,Dervallières - Zola,35 000
3,Hauts-Pavés - Saint-Félix,35 800
4,Malakoff - Saint-Donatien,34 669
5,Île de Nantes,15 818
6,Breil - Barberie,24 418
7,Nantes Nord,24 833
8,Nantes Erdre,26 738
9,Doulon - Bottière,30 147


In [11]:
# @hidden_cell
coor = pd.read_csv('//Users//caroline//Desktop//nantes_quartiers_lat-long.csv')

In [12]:
# @hidden_cell
coor.head()

,Quartier,Latitude,Longitude
0,Centre-ville,47.200142,-1.550053
1,Bellevue - Chantenay - Sainte-Anne,47.183478,-1.600042
2,Dervallières - Zola,47.216678,-1.583383
3,Hauts-Pavés - Saint-Félix,47.216794,-1.550108
4,Malakoff - Saint-Donatien,47.216736,-1.533361


In [13]:
# @hidden_cell
coor = coor.rename(columns={'Quartier': 'Neighbourhood'})

In [14]:
# @hidden_cell
coor.head()

,Neighbourhood,Latitude,Longitude
0,Centre-ville,47.200142,-1.550053
1,Bellevue - Chantenay - Sainte-Anne,47.183478,-1.600042
2,Dervallières - Zola,47.216678,-1.583383
3,Hauts-Pavés - Saint-Félix,47.216794,-1.550108
4,Malakoff - Saint-Donatien,47.216736,-1.533361


In [15]:
# @hidden_cell
neighbourhoods_df = pd.merge(neighbourhoods1, coor)

In [16]:
# @hidden_cell
neighbourhoods_df.head(13)

,Neighbourhood,Population,Latitude,Longitude
0,Centre-ville,28 485,47.200142,-1.550053
1,Bellevue - Chantenay - Sainte-Anne,25 000,47.183478,-1.600042
2,Dervallières - Zola,35 000,47.216678,-1.583383
3,Hauts-Pavés - Saint-Félix,35 800,47.216794,-1.550108
4,Malakoff - Saint-Donatien,34 669,47.216736,-1.533361
5,Île de Nantes,15 818,47.200042,-1.533472
6,Breil - Barberie,24 418,47.216831,-1.566789
7,Nantes Nord,24 833,47.250031,-1.550128
8,Nantes Erdre,26 738,47.250092,-1.516767
9,Doulon - Bottière,30 147,47.233394,-1.500089


In [17]:
# @hidden_cell
neighbourhoods_df = pd.DataFrame(neighbourhoods_df)

In [18]:
from geopy.geocoders import Nominatim

In [31]:
quartiers = ['Centre-ville','Bellevue - Chantenay - Sainte-Anne','Dervallières - Zola','Hauts-Pavés - Saint-Félix','Malakoff - Saint-Donatien','Île de Nantes','Breil - Barberie', 'Nantes Nord', 'Nantes Erdre', 'Doulon - Bottiere', 'Nantes Sud']

quartier_coord = []
for quartier in quartiers:
    address = '{}, Nantes, France'.format(quartier)
    geolocator = Nominatim(user_agent="nantes_explorer_2")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(quartier, latitude, longitude))
   
    quartier_coord = [{'Neighbourhood': quartier, 'Latitude': latitude, 'Longitude': longitude}]
    df = pd.DataFrame(quartier_coord)

The geograpical coordinates of Centre-ville are 47.214839850000004, -1.557936644670939.
The geograpical coordinates of Bellevue - Chantenay - Sainte-Anne are 47.1977826, -1.5979483854260317.
The geograpical coordinates of Dervallières - Zola are 47.2177889, -1.5889576428457481.
The geograpical coordinates of Hauts-Pavés - Saint-Félix are 47.2287293, -1.5644036887311255.
The geograpical coordinates of Malakoff - Saint-Donatien are 47.223279399999996, -1.536068311104002.
The geograpical coordinates of Île de Nantes are 47.20704805, -1.5462102466548702.
The geograpical coordinates of Breil - Barberie are 47.2351552, -1.5738850630891967.
The geograpical coordinates of Nantes Nord are 47.258410350000005, -1.5663227883277482.
The geograpical coordinates of Nantes Erdre are 47.2649793, -1.5216089444961194.
The geograpical coordinates of Doulon - Bottiere are 47.23940985, -1.5094380518266286.
The geograpical coordinates of Nantes Sud are 47.192114000000004, -1.5324685615599893.


In [32]:
df.head()

,Neighbourhood,Latitude,Longitude
0,Nantes Sud,47.192114,-1.532469


In [28]:
print(quartier_coord)

[['Nantes Sud', 47.192114000000004, -1.5324685615599893]]


In [27]:
quartier_df = pd.DataFrame(quartier_coord)
quartier_df.head()

,0,1,2
0,Nantes Sud,47.192114,-1.532469


In [23]:
address = 'Doulon, Nantes, France'

geolocator = Nominatim(user_agent="nantes_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Nantes are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Nantes are 47.2261735, -1.5223284.


In [19]:
map_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)

In [20]:
for lat, long, neighbourhood in zip(neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"], neighbourhoods_df["Neighbourhood"]):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes)  
    
map_nantes

In [21]:
# @hidden_cell
CLIENT_ID = 'D2JOFLEXDHMNPZGNMNIJR53L0ILI0UELERUPUB3XL4WREDS1'
CLIENT_SECRET = '0WFKC0ECIIPQ1C1M5OOCHONJBH1NZBTACNHDKCZT5KOUH0ES'
VERSION = '20180605'
radius = 5000
LIMIT = 100

categoryId =  
'4f4533804b9074f6e4fb0105' #elementary school  
'4bf58dd8d48988d13d941735' #high school  
'52e81612bcbc57f1066b7a46' #private school  
'52e81612bcbc57f1066b7a48' #language school  
'52e81612bcbc57f1066b7a47' #religious school  
'4bf58dd8d48988d13b941735' #school  
Nursery School
4f4533814b9074f6e4fb0107  
Preschool
52e81612bcbc57f1066b7a45

In [82]:
radius = 1000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
      
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
       
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    venues_list = []
    venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    print(venues_list)

[[('Centre-ville', 'Usine Beghin-Say', 47.20044377215564, -1.5532779693603516, 'Factory'), ('Centre-ville', 'Melting Potes', 47.20501389870853, -1.5578338518336021, 'French Restaurant'), ('Centre-ville', 'Place Mangin', 47.20071442465326, -1.5451129664393608, 'Plaza'), ('Centre-ville', 'Le Louis Blanc', 47.2073031102166, -1.5551488066699513, 'Bar'), ('Centre-ville', 'Gare de Nantes-État', 47.204205048373716, -1.5594792366027832, 'Historic Site'), ('Centre-ville', 'Place de la République', 47.20528381482166, -1.5546512603759766, 'Plaza'), ('Centre-ville', 'Gare SNCF de Rezé Pont-Rousseau', 47.193139136874365, -1.5495765209197996, 'Train Station'), ('Centre-ville', 'Jardin des Fonderies', 47.20591258452887, -1.5451938733135782, 'Garden'), ('Centre-ville', "L'Absence", 47.20755904672162, -1.5572169405069054, 'Lounge'), ('Centre-ville', 'Les Fonderies', 47.20586413817507, -1.5453354226923384, 'French Restaurant'), ('Centre-ville', 'Station Pirmil ➋➌', 47.19617556697298, -1.542779803276062,

In [ ]:
[[('Centre-ville', 'Usine Beghin-Say', 47.20044377215564, -1.5532779693603516, 'Factory'), ('Centre-ville', 'Melting Potes', 47.20501389870853, -1.5578338518336021, 'French Restaurant'), ('Centre-ville', 'Place Mangin', 47.20071442465326, -1.5451129664393608, 'Plaza'), ('Centre-ville', 'Le Louis Blanc', 47.2073031102166, -1.5551488066699513, 'Bar'), ('Centre-ville', 'Gare de Nantes-État', 47.204205048373716, -1.5594792366027832, 'Historic Site'), ('Centre-ville', 'Place de la République', 47.20528381482166, -1.5546512603759766, 'Plaza'), ('Centre-ville', 'Gare SNCF de Rezé Pont-Rousseau', 47.193139136874365, -1.5495765209197996, 'Train Station'), ('Centre-ville', 'Jardin des Fonderies', 47.20591258452887, -1.5451938733135782, 'Garden'), ('Centre-ville', "L'Absence", 47.20755904672162, -1.5572169405069054, 'Lounge'), ('Centre-ville', 'Les Fonderies', 47.20586413817507, -1.5453354226923384, 'French Restaurant'), ('Centre-ville', 'Station Pirmil ➋➌', 47.19617556697298, -1.542779803276062, 'Tram Station'), ('Centre-ville', 'Pizza Tradition', 47.20162549089935, -1.5463580983583247, 'Pizza Place'), ('Centre-ville', 'Domaine Du Vigneron', 47.20371603, -1.55240992, 'Wine Shop'), ('Centre-ville', "Marché d'Intérêt National de Nantes", 47.20155201181927, -1.5566359428552599, 'Food & Drink Shop'), ('Centre-ville', 'ibis Styles', 47.1927490906441, -1.5553218126296997, 'Hotel'), ('Centre-ville', 'Arrêt Pirmil [C4,24,27,28,29,36,39,42,98,Lila]', 47.19625315569016, -1.5427095131011976, 'Bus Stop'), ('Centre-ville', 'Mangin Beaulieu', 47.2047521476517, -1.543295704680286, 'Basketball Stadium'), ('Centre-ville', 'Station Gare de Pont Rousseau ➋', 47.1929791016337, -1.5490490845055636, 'Tram Station'), ('Centre-ville', 'Rent A Car NANTES (ILE DE NANTES)', 47.200472, -1.561166, 'Rental Car Location'), ('Centre-ville', 'ibis Budget', 47.19309174823302, -1.5553218126296997, 'Hotel'), ('Centre-ville', 'Lulu La Nantaise', 47.204773050038476, -1.5598505483299236, 'French Restaurant'), ('Centre-ville', 'Station Pont-Rousseau-Martyrs ➌', 47.19187056480316, -1.5483078360557556, 'Tram Station')]]
[[('Bellevue - Chantenay - Sainte-Anne', 'TOTAL ACCESS', 47.18538562, -1.6023049, 'Gas Station'), ('Bellevue - Chantenay - Sainte-Anne', 'Samia', 47.1792693, -1.59675996, 'Other Repair Shop'), ('Bellevue - Chantenay - Sainte-Anne', 'Arrêt Châtelier [36]', 47.184786, -1.593201, 'Bus Stop'), ('Bellevue - Chantenay - Sainte-Anne', 'Lariviere', 47.188266, -1.591918, 'Furniture / Home Store'), ('Bellevue - Chantenay - Sainte-Anne', 'Station Neustrie ➌', 47.177291, -1.592734, 'Tram Station'), ('Bellevue - Chantenay - Sainte-Anne', 'Arrêt Neustrie [36,37,74,88,99]', 47.177149, -1.592653, 'Bus Stop'), ('Bellevue - Chantenay - Sainte-Anne', "Bières'e Choppes", 47.18732997, -1.58874899, 'Liquor Store')]]
[[('Dervallières - Zola', 'Le Concorde', 47.211221591307456, -1.5867507942322672, 'Movie Theater'), ('Dervallières - Zola', 'Parc de Procé', 47.22367, -1.582348, 'Park'), ('Dervallières - Zola', 'Intermarché Express', 47.21505, -1.58637, 'Supermarket'), ('Dervallières - Zola', 'Place du Général Mellinet', 47.211355111362266, -1.576361060142517, 'Plaza'), ('Dervallières - Zola', 'Place Canclaux', 47.214350417010735, -1.574467420578003, 'Plaza'), ('Dervallières - Zola', 'Central Form', 47.21364213106233, -1.5774236318698376, 'Gym'), ('Dervallières - Zola', 'Place Zola', 47.213983794692936, -1.5863547159426459, 'Plaza'), ('Dervallières - Zola', "Boulangerie d'Honoré", 47.21248474771768, -1.5728366374969482, 'Bakery'), ('Dervallières - Zola', 'Carrefour City', 47.2128162, -1.5715732, 'Supermarket'), ('Dervallières - Zola', '[C3] Lamartine', 47.21646919697718, -1.5791685341803248, 'Bus Stop'), ('Dervallières - Zola', 'Picard', 47.21429328, -1.58634188, 'Grocery Store'), ('Dervallières - Zola', 'Pharmacie Poincare', 47.22132, -1.58582, 'Pharmacy'), ('Dervallières - Zola', 'Bowl de Procé', 47.22062086138098, -1.5781771307155983, 'Skate Park'), ('Dervallières - Zola', 'Le Canclaux', 47.214337, -1.575037, 'Bistro'), ('Dervallières - Zola', 'Le Loup, Le Homard et la Crevette', 47.210416532689834, -1.5868471261213828, 'Seafood Restaurant'), ('Dervallières - Zola', "Boulangerie L'Artisan", 47.2135913684007, -1.5738272582296937, 'Bakery'), ('Dervallières - Zola', 'Tabac-Presse Canclaux', 47.213498364315164, -1.5737771235303377, 'Smoke Shop'), ('Dervallières - Zola', 'Jardins ouvriers de Zola', 47.21232861797396, -1.5931929636455788, 'Park'), ('Dervallières - Zola', 'Station Égalité ➊', 47.209354, -1.588612, 'Tram Station'), ('Dervallières - Zola', 'Complexe Sportif Des dervallieres', 47.22099854048025, -1.5940003295547613, 'Track Stadium'), ('Dervallières - Zola', 'Boulangerie Emma', 47.224365, -1.576851, 'Bakery')]]
[[('Hauts-Pavés - Saint-Félix', 'Château des Ducs de Bretagne', 47.21593138004272, -1.5496145648598079, 'Castle'), ('Hauts-Pavés - Saint-Félix', 'La Compagnie du Café Théatre', 47.216831038300334, -1.5509578487147495, 'Comedy Club'), ('Hauts-Pavés - Saint-Félix', 'Kit Kat Bar', 47.2156591547589, -1.552548698894604, 'Cocktail Bar'), ('Hauts-Pavés - Saint-Félix', 'Cathédrale Saint-Pierre (Cathédrale Saint-Pierre-et-Saint-Paul)', 47.21836575599015, -1.5502095222473145, 'Church'), ('Hauts-Pavés - Saint-Félix', 'Basilic & Co', 47.21627134092316, -1.5514524073406626, 'Pizza Place'), ('Hauts-Pavés - Saint-Félix', 'Hôtel Okko Nantes Château', 47.21660525726646, -1.5517900457078042, 'Hotel'), ('Hauts-Pavés - Saint-Félix', 'Le Vanola', 47.21726478276263, -1.5462066561475103, 'Asian Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Le Barablabla', 47.2159393483422, -1.5526021378480193, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Le Fou du Roi', 47.216767368871565, -1.5497726687459128, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', "Le Couch'Tard", 47.216641918056226, -1.5530977019437155, 'Kebab Restaurant'), ('Hauts-Pavés - Saint-Félix', 'ABC Terroirs', 47.21814457023353, -1.5514217330270332, 'Gourmet Shop'), ('Hauts-Pavés - Saint-Félix', 'Aux Quatre Saisons', 47.2149288378335, -1.554017726468736, 'Chinese Restaurant'), ('Hauts-Pavés - Saint-Félix', "Brady's Irish Pub", 47.216494814811085, -1.5465887440481594, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Oncle Pho', 47.21685793302002, -1.5555370599031446, 'Vietnamese Restaurant'), ('Hauts-Pavés - Saint-Félix', 'La Prison du Bouffay', 47.21472271657559, -1.5531821067450131, 'Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Au Vieux Quimper', 47.21527320671112, -1.5532980859279633, 'Creperie'), ('Hauts-Pavés - Saint-Félix', 'Place du Bouffay', 47.21473590871554, -1.5531872568757554, 'Plaza'), ('Hauts-Pavés - Saint-Félix', 'Le Sur Mesure', 47.21485717759568, -1.555411358345894, 'Bar'), ('Hauts-Pavés - Saint-Félix', "Café Cult'", 47.215664855355115, -1.5550714649051698, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', "L'Épicerie", 47.21482061690571, -1.5534758394679848, 'Gastropub'), ('Hauts-Pavés - Saint-Félix', 'La Maison Café', 47.22021106439959, -1.5495960208075463, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Les Fils à Maman', 47.214918850077815, -1.5554076433181763, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Poivre Rouge', 47.21648193035155, -1.5553210633613554, 'Steakhouse'), ('Hauts-Pavés - Saint-Félix', 'Kusadasi', 47.21665832424736, -1.5564081797390197, 'Greek Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Delirium Café', 47.213444214269614, -1.550356108042254, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'La Bodega', 47.214965247374224, -1.5551051281399657, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Shaan Tandoori', 47.2149000134467, -1.555217446409087, 'Indian Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Pizzéria Pinocchio', 47.21809464406659, -1.5555682205449177, 'Pizza Place'), ('Hauts-Pavés - Saint-Félix', 'Inde Et Vous', 47.21946031216918, -1.5559944207133096, 'Indian Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Jardin des Plantes', 47.219218338742124, -1.542506217956543, 'Botanical Garden'), ('Hauts-Pavés - Saint-Félix', 'Le Labo', 47.2192517833411, -1.5549601784693263, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Sugar Blue', 47.215108, -1.558233, 'Diner'), ('Hauts-Pavés - Saint-Félix', 'Le Coq en Pâte', 47.218483656221665, -1.5571459877605462, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Le Fournil Bossard', 47.22087096, -1.5556138, 'Bakery'), ('Hauts-Pavés - Saint-Félix', 'Madam Blå', 47.21670736673864, -1.5564031966050442, 'Tea Room'), ('Hauts-Pavés - Saint-Félix', 'Le Lieu Unique', 47.21544355211624, -1.5455693006515503, 'Concert Hall'), ('Hauts-Pavés - Saint-Félix', 'Plan B', 47.21163752270686, -1.5466958284378052, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', "Musée d'arts de Nantes", 47.2195681123155, -1.5473127365112305, 'Art Museum'), ('Hauts-Pavés - Saint-Félix', "L'u.ni", 47.211781380147414, -1.5470522064236525, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Place Royale', 47.214449203212176, -1.558389703315014, 'Plaza'), ('Hauts-Pavés - Saint-Félix', 'Marché de Talensac', 47.22135058981144, -1.5577092455767456, 'Farmers Market'), ('Hauts-Pavés - Saint-Félix', 'Passage Pommeraye', 47.21355643818724, -1.5597593631279587, 'Historic Site'), ('Hauts-Pavés - Saint-Félix', "Bords de l'Erdre", 47.223466859247864, -1.5532881877331859, 'Trail'), ('Hauts-Pavés - Saint-Félix', 'Maison Larnicol', 47.213302513745674, -1.5590291045611773, 'Gourmet Shop'), ('Hauts-Pavés - Saint-Félix', 'La Scierie', 47.211432271142854, -1.556740161765205, 'Lounge'), ('Hauts-Pavés - Saint-Félix', 'French Coffee Shop', 47.2128, -1.5587, 'Café'), ('Hauts-Pavés - Saint-Félix', 'Place Graslin', 47.21314534053496, -1.5621120978549354, 'Plaza'), ('Hauts-Pavés - Saint-Félix', 'La passerelle de Marcel', 47.21249880242966, -1.5394487408888933, 'Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Shalimar', 47.21300866524362, -1.561808649502115, 'Indian Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Le B', 47.2128215, -1.5616199, 'Burger Joint'), ('Hauts-Pavés - Saint-Félix', 'Les Pieds dans Le Plat', 47.212709402221144, -1.560992181541278, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Rue Crébillon', 47.213472125899386, -1.5612990623182705, 'Pedestrian Plaza'), ('Hauts-Pavés - Saint-Félix', 'Rue Santeuil', 47.21314794236804, -1.56030535697937, 'Pedestrian Plaza'), ('Hauts-Pavés - Saint-Félix', 'Le Song', 47.21336200764077, -1.5604149521409663, 'Asian Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Rue Suffren', 47.21303680318365, -1.5609517693519592, 'Pedestrian Plaza'), ('Hauts-Pavés - Saint-Félix', 'Nespresso Boutique', 47.21419037109422, -1.5606418393603263, 'Coffee Shop'), ('Hauts-Pavés - Saint-Félix', 'Place Aristide Briand', 47.21725081939781, -1.5627998113632202, 'Plaza'), ('Hauts-Pavés - Saint-Félix', 'Hôtel Pommeraye', 47.214167990614484, -1.560433514269721, 'Hotel'), ('Hauts-Pavés - Saint-Félix', 'Radisson Blu Hotel', 47.2175975, -1.5631576, 'Hotel'), ('Hauts-Pavés - Saint-Félix', 'Subway', 47.21877383149653, -1.560906171798706, 'Sandwich Place'), ('Hauts-Pavés - Saint-Félix', 'Théâtre Graslin', 47.213519619259884, -1.5622365474700928, 'Opera House'), ('Hauts-Pavés - Saint-Félix', 'Aux Petits Oignons', 47.213093, -1.560985, 'Restaurant'), ('Hauts-Pavés - Saint-Félix', "L'Univers", 47.21305172569131, -1.5618894851478058, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Stade Marcel Saupin', 47.21319860619315, -1.5382321589487937, 'Soccer Stadium'), ('Hauts-Pavés - Saint-Félix', '…Et la Fourmi', 47.21337307460104, -1.5609843496418427, 'Diner'), ('Hauts-Pavés - Saint-Félix', 'Intermarché Express', 47.223935321881314, -1.5505714477838348, 'Supermarket'), ('Hauts-Pavés - Saint-Félix', 'Le Corneille', 47.21395923825291, -1.5625173218583173, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Kyriad Nantes Centre Graslin', 47.21185, -1.5610277778, 'Hotel'), ('Hauts-Pavés - Saint-Félix', 'Le Palatium', 47.21743158412126, -1.5626365879446504, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Café Charbon Le Guingois', 47.21317841176193, -1.5603983585269827, 'French Restaurant'), ('Hauts-Pavés - Saint-Félix', "Pickwick's", 47.213177409975444, -1.5607686485286592, 'Bar'), ('Hauts-Pavés - Saint-Félix', 'Ibis Tour de Bretagne', 47.2191062, -1.5604114, 'Hotel'), ('Hauts-Pavés - Saint-Félix', 'Zara', 47.21568760283227, -1.5605609688828437, 'Clothing Store'), ('Hauts-Pavés - Saint-Félix', "Le Fournil de l'Erdre", 47.2234531138611, -1.5515335935869121, 'Bakery'), ('Hauts-Pavés - Saint-Félix', 'Station Saint-Mihiel ➋', 47.223315, -1.553399, 'Tram Station'), ('Hauts-Pavés - Saint-Félix', 'Station Jean Jaurès ➌', 47.218632, -1.560422, 'Tram Station'), ('Hauts-Pavés - Saint-Félix', 'Eurodif', 47.21534334900303, -1.5606245398521423, 'Clothing Store'), ('Hauts-Pavés - Saint-Félix', 'Song Comptoir', 47.21343580710544, -1.5603145455954344, 'Asian Restaurant'), ('Hauts-Pavés - Saint-Félix', 'OXYBUL Nantes', 47.21548809999999, -1.5618474, 'Toy / Game Store'), ('Hauts-Pavés - Saint-Félix', 'Arrêt Chanzy [C1]', 47.22501136587666, -1.5481204365780745, 'Bus Stop'), ('Hauts-Pavés - Saint-Félix', 'Vivre & Savourer', 47.21467726656559, -1.5378431267904564, 'Restaurant'), ('Hauts-Pavés - Saint-Félix', 'Oceania - Hotel de france', 47.213252, -1.561878, 'Resort'), ('Hauts-Pavés - Saint-Félix', 'Station Manufacture ➊', 47.21875561390695, -1.5373778343200684, 'Tram Station'), ('Hauts-Pavés - Saint-Félix', 'Mauricette', 47.22060506544739, -1.561919573969105, 'Wine Bar')]]
[[('Malakoff - Saint-Donatien', 'La passerelle de Marcel', 47.21249880242966, -1.5394487408888933, 'Restaurant'), ('Malakoff - Saint-Donatien', 'Jardin des Plantes', 47.219218338742124, -1.542506217956543, 'Botanical Garden'), ('Malakoff - Saint-Donatien', 'Petite Amazonie', 47.217149, -1.527679, 'Park'), ('Malakoff - Saint-Donatien', 'Hotel Sozo', 47.21999492989357, -1.5410553147467687, 'Hotel'), ('Malakoff - Saint-Donatien', 'Le Vanola', 47.21726478276263, -1.5462066561475103, 'Asian Restaurant'), ('Malakoff - Saint-Donatien', 'Intermarché Super', 47.21467, -1.52992, 'Supermarket'), ('Malakoff - Saint-Donatien', "Brady's Irish Pub", 47.216494814811085, -1.5465887440481594, 'Bar'), ('Malakoff - Saint-Donatien', 'Écluse Saint-Félix', 47.21195543007231, -1.541166612604851, 'Canal Lock'), ('Malakoff - Saint-Donatien', 'Le Lieu Unique', 47.21544355211624, -1.5455693006515503, 'Concert Hall'), ('Malakoff - Saint-Donatien', 'Stade Marcel Saupin', 47.21319860619315, -1.5382321589487937, 'Soccer Stadium'), ('Malakoff - Saint-Donatien', 'Le Voyage à Nantes', 47.214621861809746, -1.5461726942843153, 'Tourist Information Center'), ('Malakoff - Saint-Donatien', 'Canal Saint-Félix', 47.21499172567142, -1.5435361862182617, 'Canal'), ('Malakoff - Saint-Donatien', 'Brasserie Félix', 47.214277261180314, -1.5434769528291392, 'French Restaurant'), ('Malakoff - Saint-Donatien', "La Boulangerie d'Honoré", 47.213688, -1.545368, 'Bakery'), ('Malakoff - Saint-Donatien', 'Kyriad Nantes Centre Gare', 47.2175, -1.5448388889, 'Hotel'), ('Malakoff - Saint-Donatien', 'Tour LU', 47.21544355211624, -1.5462398529052734, 'Historic Site'), ('Malakoff - Saint-Donatien', 'Hôtel Mercure Nantes Centre Gare', 47.216185762031095, -1.543229265970922, 'Hotel'), ('Malakoff - Saint-Donatien', 'Carrefour City', 47.21384392343942, -1.5455934405326843, 'Supermarket'), ('Malakoff - Saint-Donatien', 'Paul', 47.2175811, -1.5419391, 'Sandwich Place'), ('Malakoff - Saint-Donatien', 'Super U', 47.224290053699214, -1.5306338168786928, 'Supermarket'), ('Malakoff - Saint-Donatien', 'Station Moutonnerie ➊', 47.219575, -1.533139, 'Tram Station'), ('Malakoff - Saint-Donatien', 'Gare SNCF de Nantes', 47.21713422336746, -1.5417015552520752, 'Train Station'), ('Malakoff - Saint-Donatien', 'Côté Ardoise', 47.213556278409534, -1.539302989567666, 'Pizza Place'), ('Malakoff - Saint-Donatien', 'Vitalliance Nantes Sud - Aide à domicile', 47.21599, -1.525233, 'Home Service'), ('Malakoff - Saint-Donatien', 'Novotel Palais des Congrès', 47.21196200552412, -1.5411594078767812, 'Hotel'), ('Malakoff - Saint-Donatien', 'Station Hôpital Bellier ➊', 47.221189428428964, -1.5245139598846436, 'Tram Station'), ('Malakoff - Saint-Donatien', 'Bicloo Région Nº84', 47.210850297382336, -1.5258889255730124, 'Bike Rental / Bike Share'), ('Malakoff - Saint-Donatien', 'Arrêt Pompidou [24,52]', 47.209314, -1.527902, 'Bus Stop'), ('Malakoff - Saint-Donatien', 'Le Fournil Mitronette', 47.20844729645882, -1.5294637373618738, 'Bakery'), ('Malakoff - Saint-Donatien', 'Sale & Pepe', 47.21329677285771, -1.5452729165554047, 'Pizza Place')]]
[[('Île de Nantes', 'SEPHORA', 47.205272, -1.534314, 'Cosmetics Shop'), ('Île de Nantes', 'Place Mangin', 47.20071442465326, -1.5451129664393608, 'Plaza'), ('Île de Nantes', 'Starbucks', 47.20515986060009, -1.5370003550389046, 'Coffee Shop'), ('Île de Nantes', 'Campanile Nantes Centre - Saint Jacques', 47.19624247189448, -1.5309141911424164, 'Hotel'), ('Île de Nantes', 'Arrêt Gréneraie [C4,27,28,29,36,39,98,Express]', 47.199498338000296, -1.5334820586285856, 'Bus Stop'), ('Île de Nantes', 'Station Gréneraie ➍', 47.199689925919365, -1.53354823002027, 'Bus Stop'), ('Île de Nantes', 'Hôtel Mercure Île de Nantes', 47.2065243810356, -1.5282858884541979, 'Hotel'), ('Île de Nantes', 'C.C Beaulieu', 47.2055925112074, -1.535066581962283, 'Shopping Mall'), ('Île de Nantes', 'Station Pirmil ➋➌', 47.19617556697298, -1.542779803276062, 'Tram Station'), ('Île de Nantes', 'Station Bonne Garde ➍', 47.19683077644586, -1.5313092979718468, 'Bus Stop'), ('Île de Nantes', 'Desigual Nantes', 47.20413215795766, -1.534566879272461, 'Clothing Store'), ('Île de Nantes', 'H&M', 47.20438798579326, -1.5366987744346263, 'Clothing Store'), ('Île de Nantes', 'Super U', 47.19437417974542, -1.5298375941046387, 'Supermarket'), ('Île de Nantes', 'Station Clos Toreau ➍', 47.19430249277148, -1.5289978596119482, 'Bus Stop'), ('Île de Nantes', 'Station Île de Nantes ➍', 47.20610350931599, -1.5387995680636681, 'Bus Stop'), ('Île de Nantes', 'Arrêt Pirmil [C4,24,27,28,29,36,39,42,98,Lila]', 47.19625315569016, -1.5427095131011976, 'Bus Stop'), ('Île de Nantes', 'ASFORA', 47.20384364, -1.52408169, 'Miscellaneous Shop'), ('Île de Nantes', 'Mangin Beaulieu', 47.2047521476517, -1.543295704680286, 'Basketball Stadium'), ('Île de Nantes', 'SEVEN URBAN SUITES**** NANTES CENTRE', 47.2078212720399, -1.5394270260392162, 'Hotel'), ('Île de Nantes', 'Le Fournil Mitronette', 47.20844729645882, -1.5294637373618738, 'Bakery'), ('Île de Nantes', 'Pizza Tradition', 47.20162549089935, -1.5463580983583247, 'Pizza Place')]]
[[('Breil - Barberie', 'Vincent Guerlais', 47.21466674634973, -1.5634316921269844, 'Dessert Shop'), ('Breil - Barberie', 'Place Graslin', 47.21314534053496, -1.5621120978549354, 'Plaza'), ('Breil - Barberie', 'Shalimar', 47.21300866524362, -1.561808649502115, 'Indian Restaurant'), ('Breil - Barberie', 'Place Delorme', 47.215042738527316, -1.563577651977539, 'Plaza'), ('Breil - Barberie', 'Passage Pommeraye', 47.21355643818724, -1.5597593631279587, 'Historic Site'), ('Breil - Barberie', "Place de l'Édit de Nantes", 47.21454718299273, -1.567472219467163, 'Plaza'), ('Breil - Barberie', 'Le B', 47.2128215, -1.5616199, 'Burger Joint'), ('Breil - Barberie', 'Le Café Kléber', 47.214458973511, -1.5661168320228478, 'Bar'), ('Breil - Barberie', 'Sugar Blue', 47.215108, -1.558233, 'Diner'), ('Breil - Barberie', 'Place Royale', 47.214449203212176, -1.558389703315014, 'Plaza'), ('Breil - Barberie', "Muséum d'Histoire Naturelle de Nantes", 47.212710672101394, -1.5648972988128662, 'Science Museum'), ('Breil - Barberie', 'Place Aristide Briand', 47.21725081939781, -1.5627998113632202, 'Plaza'), ('Breil - Barberie', 'Les Pieds dans Le Plat', 47.212709402221144, -1.560992181541278, 'French Restaurant'), ('Breil - Barberie', 'Maison Larnicol', 47.213302513745674, -1.5590291045611773, 'Gourmet Shop'), ('Breil - Barberie', 'Le Coq en Pâte', 47.218483656221665, -1.5571459877605462, 'French Restaurant'), ('Breil - Barberie', 'Place de la Monnaie — Square Docteur Louis Bureau', 47.21301129579702, -1.5651172399520874, 'Plaza'), ('Breil - Barberie', 'Marché de Talensac', 47.22135058981144, -1.5577092455767456, 'Farmers Market'), ('Breil - Barberie', 'Kusadasi', 47.21665832424736, -1.5564081797390197, 'Greek Restaurant'), ('Breil - Barberie', 'Radisson Blu Hotel', 47.2175975, -1.5631576, 'Hotel'), ('Breil - Barberie', 'Oncle Pho', 47.21685793302002, -1.5555370599031446, 'Vietnamese Restaurant'), ('Breil - Barberie', 'Katorza', 47.213583142463634, -1.5625764000234976, 'Indie Movie Theater'), ('Breil - Barberie', 'Rue Crébillon', 47.213472125899386, -1.5612990623182705, 'Pedestrian Plaza'), ('Breil - Barberie', 'French Coffee Shop', 47.2128, -1.5587, 'Café'), ('Breil - Barberie', 'Le Sur Mesure', 47.21485717759568, -1.555411358345894, 'Bar'), ('Breil - Barberie', "Tabl'o Gourmand", 47.21739339312881, -1.5571402324736385, 'Breakfast Spot'), ('Breil - Barberie', 'Les Fils à Maman', 47.214918850077815, -1.5554076433181763, 'French Restaurant'), ('Breil - Barberie', 'Inde Et Vous', 47.21946031216918, -1.5559944207133096, 'Indian Restaurant'), ('Breil - Barberie', 'Madam Blå', 47.21670736673864, -1.5564031966050442, 'Tea Room'), ('Breil - Barberie', 'Les Fées Maison', 47.21564821686454, -1.5569775741191707, 'Bar'), ('Breil - Barberie', 'PitaPit', 47.21355487851994, -1.5579202108522396, 'Fast Food Restaurant'), ('Breil - Barberie', 'Le Fournil Bossard', 47.22087096, -1.5556138, 'Bakery'), ('Breil - Barberie', "Café Cult'", 47.215664855355115, -1.5550714649051698, 'French Restaurant'), ('Breil - Barberie', 'Rue Suffren', 47.21303680318365, -1.5609517693519592, 'Pedestrian Plaza'), ('Breil - Barberie', 'Aux Quatre Saisons', 47.2149288378335, -1.554017726468736, 'Chinese Restaurant'), ('Breil - Barberie', 'Pizzéria Pinocchio', 47.21809464406659, -1.5555682205449177, 'Pizza Place'), ('Breil - Barberie', 'Poivre Rouge', 47.21648193035155, -1.5553210633613554, 'Steakhouse'), ('Breil - Barberie', 'Shaan Tandoori', 47.2149000134467, -1.555217446409087, 'Indian Restaurant'), ('Breil - Barberie', 'Rue Santeuil', 47.21314794236804, -1.56030535697937, 'Pedestrian Plaza'), ('Breil - Barberie', 'La Bodega', 47.214965247374224, -1.5551051281399657, 'Bar'), ('Breil - Barberie', 'Le Song', 47.21336200764077, -1.5604149521409663, 'Asian Restaurant'), ('Breil - Barberie', 'machin machine', 47.209131345461465, -1.5684473924495244, 'French Restaurant'), ('Breil - Barberie', 'Place Canclaux', 47.214350417010735, -1.574467420578003, 'Plaza'), ('Breil - Barberie', "L'industrie Brasserie", 47.209720778817214, -1.5706875724465066, 'French Restaurant'), ('Breil - Barberie', 'Le Labo', 47.2192517833411, -1.5549601784693263, 'Bar'), ('Breil - Barberie', 'Théâtre Graslin', 47.213519619259884, -1.5622365474700928, 'Opera House'), ('Breil - Barberie', 'La Scierie', 47.211432271142854, -1.556740161765205, 'Lounge'), ('Breil - Barberie', 'Marché de la Petite Hollande', 47.211098431991196, -1.5599283167503548, 'Farmers Market'), ('Breil - Barberie', '92,5 Coffee Shop', 47.21445256801083, -1.5556121943013228, 'Coffee Shop'), ('Breil - Barberie', 'Esperance Café', 47.208182, -1.563151, 'Coffee Shop'), ('Breil - Barberie', 'Nespresso Boutique', 47.21419037109422, -1.5606418393603263, 'Coffee Shop'), ('Breil - Barberie', 'Square de La Bourse', 47.21245924006551, -1.5589267015457153, 'Plaza'), ('Breil - Barberie', 'Yum Yum', 47.21408165662549, -1.5545975522162758, 'Chinese Restaurant'), ('Breil - Barberie', 'Rue Jean-Jacques Rousseau', 47.212325228613864, -1.5599610652312303, 'Pedestrian Plaza'), ('Breil - Barberie', 'Église Sainte-Croix', 47.21537796464537, -1.5541470050811768, 'Church'), ('Breil - Barberie', 'Subway', 47.21877383149653, -1.560906171798706, 'Sandwich Place'), ('Breil - Barberie', 'Place de la Bourse', 47.21279812644316, -1.5590393543243408, 'Plaza'), ('Breil - Barberie', 'Hôtel Pommeraye', 47.214167990614484, -1.560433514269721, 'Hotel'), ('Breil - Barberie', 'Cony Bagel Store', 47.213896588841294, -1.5635823807728289, 'Bagel Shop'), ('Breil - Barberie', 'Café Le Flesselles', 47.21430122540117, -1.5546861290931702, 'Bar'), ('Breil - Barberie', 'Aux Petits Oignons', 47.213093, -1.560985, 'Restaurant'), ('Breil - Barberie', "Square des Combattants d'Afrique du Nord", 47.209269, -1.570739, 'Park'), ('Breil - Barberie', 'Place du Général Mellinet', 47.211355111362266, -1.576361060142517, 'Plaza'), ('Breil - Barberie', "L'Univers", 47.21305172569131, -1.5618894851478058, 'Bar'), ('Breil - Barberie', 'Carrefour City', 47.2128162, -1.5715732, 'Supermarket'), ('Breil - Barberie', 'Square Jean-Baptiste Daviais', 47.211916288559266, -1.5579020977020264, 'Park'), ('Breil - Barberie', 'Place René Bouhier', 47.20958772879432, -1.5705084800720215, 'Plaza'), ('Breil - Barberie', 'Place Viarme', 47.220974470267166, -1.562579870223999, 'Plaza'), ('Breil - Barberie', 'Cours Cambronne', 47.21194179647257, -1.5631914138793945, 'Park'), ('Breil - Barberie', 'Au Chat Noir', 47.21307154275112, -1.5562914899725888, 'Bar'), ('Breil - Barberie', 'Île Feydeau', 47.21278453937351, -1.555472851911945, 'Historic Site'), ('Breil - Barberie', "Boulangerie d'Honoré", 47.21248474771768, -1.5728366374969482, 'Bakery'), ('Breil - Barberie', 'Parc des Capucins', 47.22530989860401, -1.5627515316009521, 'Park'), ('Breil - Barberie', 'Un coin en ville', 47.21291717813463, -1.558993961048719, 'French Restaurant'), ('Breil - Barberie', 'Musée Dobrée', 47.212641, -1.56648, 'Art Museum'), ('Breil - Barberie', 'La Cigale', 47.21275439929031, -1.5620890259742737, 'Bistro'), ('Breil - Barberie', 'Le Palatium', 47.21743158412126, -1.5626365879446504, 'Bar'), ('Breil - Barberie', 'Central Form', 47.21364213106233, -1.5774236318698376, 'Gym'), ('Breil - Barberie', 'Le Corneille', 47.21395923825291, -1.5625173218583173, 'Bar'), ('Breil - Barberie', 'Carrefour City', 47.2236, -1.5636, 'Supermarket'), ('Breil - Barberie', 'Allée de Turenne', 47.21229890597089, -1.5556973218917847, 'Pedestrian Plaza'), ('Breil - Barberie', '…Et la Fourmi', 47.21337307460104, -1.5609843496418427, 'Diner'), ('Breil - Barberie', "Mémorial de l'Abolition de l'Esclavage", 47.20932899362073, -1.5651977062225342, 'Historic Site'), ('Breil - Barberie', 'Le Wok', 47.2130055807206, -1.5628668973510855, 'Asian Restaurant'), ('Breil - Barberie', 'Le Loft', 47.214406226044765, -1.5634530316180835, 'Nightclub'), ('Breil - Barberie', 'Place du Commerce', 47.21327234283442, -1.557903074184651, 'Plaza'), ('Breil - Barberie', 'Kyriad Nantes Centre Graslin', 47.21185, -1.5610277778, 'Hotel'), ('Breil - Barberie', 'Sushi Shop', 47.21339535233658, -1.563063884198319, 'Sushi Restaurant'), ('Breil - Barberie', 'Café Charbon Le Guingois', 47.21317841176193, -1.5603983585269827, 'French Restaurant'), ('Breil - Barberie', 'Au Vestiaire', 47.21223224694089, -1.55724827984508, 'Lounge'), ('Breil - Barberie', 'Aspendos', 47.210516057932175, -1.5627161476716172, 'Greek Restaurant'), ('Breil - Barberie', 'Piscine Léo-Lagrange', 47.21018168222853, -1.5609805493114546, 'Gym Pool'), ('Breil - Barberie', "Pickwick's", 47.213177409975444, -1.5607686485286592, 'Bar'), ('Breil - Barberie', 'Zara', 47.21568760283227, -1.5605609688828437, 'Clothing Store'), ('Breil - Barberie', 'Ibis Tour de Bretagne', 47.2191062, -1.5604114, 'Hotel'), ('Breil - Barberie', 'Bahiana (Rhumerie)', 47.21229406847856, -1.559149556828387, 'Cocktail Bar'), ('Breil - Barberie', 'Subway', 47.213116, -1.558937, 'Sandwich Place'), ('Breil - Barberie', 'Fnac', 47.21287084, -1.55847558, 'Department Store'), ('Breil - Barberie', 'Soon Café', 47.21321340943276, -1.558267639422741, 'Bar'), ('Breil - Barberie', 'Les Arcades', 47.21390144973714, -1.563744068654267, 'Bar'), ('Breil - Barberie', 'Le Marceau', 47.21577713, -1.56394079, 'French Restaurant')]]
[[('Nantes Nord', 'La Belle Equipe', 47.25364165879284, -1.5404125855450665, 'Bar'), ('Nantes Nord', 'Piscine du Petit-Port', 47.24300338120784, -1.5550773885334723, 'Gym Pool'), ('Nantes Nord', 'Parc du Petit Port', 47.245648887666, -1.5580415725708008, 'Park'), ('Nantes Nord', 'Patinoire du Petit-Port', 47.242907838041035, -1.5555886118890905, 'Skating Rink'), ('Nantes Nord', 'Station École Centrale Audencia ➋', 47.248651, -1.551235, 'Tram Station'), ('Nantes Nord', 'Stadium Metropolitain Pierre-Quinon', 47.24859272017214, -1.5554068451644143, 'Track Stadium'), ('Nantes Nord', 'Station Facultés ➋', 47.24575358557779, -1.5552118420600891, 'Tram Station'), ('Nantes Nord', 'Arrêt La Jonelière [51]', 47.25236500043222, -1.5411096543620015, 'Bus Stop'), ('Nantes Nord', 'Match Hockey Nantes', 47.2430010807037, -1.5554977813785793, 'Hockey Arena'), ('Nantes Nord', 'Navibus Passeur Port Boyer', 47.242051, -1.545993, 'Pier'), ('Nantes Nord', 'Bowling du Petit Port', 47.24238585076487, -1.5560245513916016, 'Bowling Alley')]]
[[('Nantes Erdre', 'Gare SNCF de Haluchère Batignolles', 47.249046, -1.522481, 'Train Station'), ('Nantes Erdre', 'Subway', 47.24827740732199, -1.5132858196139256, 'Sandwich Place'), ('Nantes Erdre', 'Stade de la Beaujoire', 47.25604847642898, -1.524825096130371, 'Soccer Stadium'), ('Nantes Erdre', 'Decathlon', 47.25152439648254, -1.5130287408828735, 'Sporting Goods Shop'), ('Nantes Erdre', 'E.Leclerc Hyper', 47.24893672, -1.51513598, 'Supermarket'), ('Nantes Erdre', 'Piscine Jules Verne', 47.24858708712075, -1.5178584324263487, 'Pool'), ('Nantes Erdre', 'Carrefour', 47.257927, -1.512812, 'Supermarket'), ('Nantes Erdre', 'Pizza Del Arte', 47.249722, -1.519167, 'Pizza Place'), ('Nantes Erdre', 'Ange', 47.2503701344585, -1.5188973189680224, 'Bakery'), ('Nantes Erdre', 'Flunch', 47.250789156021575, -1.5200559118426622, 'Cafeteria'), ('Nantes Erdre', 'Poivre Rouge', 47.25113384, -1.52035654, 'Restaurant'), ('Nantes Erdre', 'Chez Tante Yvette', 47.24834753360937, -1.5134444547000023, 'French Restaurant'), ('Nantes Erdre', 'Leon De Bruxelles', 47.24830386017468, -1.5133609522194238, 'Eastern European Restaurant'), ('Nantes Erdre', 'Boulanger Paridis', 47.248266085114146, -1.5133846961112583, 'Electronics Store'), ('Nantes Erdre', 'Espace Culturel Paridis', 47.24829529837074, -1.513272383585916, 'Bookstore'), ('Nantes Erdre', 'Le Puccini', 47.248251, -1.513038, 'Pizza Place'), ('Nantes Erdre', 'Ibis', 47.253791715378185, -1.515712184191898, 'Hotel'), ('Nantes Erdre', 'Arrêt Haluchère – Batignolles [C1,52,78,83,85,95,Express]', 47.24895542090911, -1.5224432945251465, 'Bus Stop'), ('Nantes Erdre', 'Station Haluchère – Batignolles ➊', 47.249032, -1.522653, 'Tram Station'), ('Nantes Erdre', 'Quality Suite Hotel', 47.25075152158686, -1.5079894746468547, 'Hotel'), ('Nantes Erdre', 'Planet Sushi Paridis', 47.24749436295154, -1.52230288298278, 'Japanese Restaurant'), ('Nantes Erdre', 'Le Bistrot Du Pr. Auvigné', 47.254907, -1.517852, 'Café'), ('Nantes Erdre', "McDonald's", 47.25707968572021, -1.5101258248603946, 'Fast Food Restaurant'), ('Nantes Erdre', "Domino's Pizza", 47.244519535961544, -1.5256255488401393, 'Pizza Place'), ('Nantes Erdre', 'CERISE Nantes La Beaujoire', 47.25354131651515, -1.5284055054311991, 'Hotel')]]
[[('Doulon - Bottière', 'Place du Vieux-Doulon', 47.23621593820584, -1.505099512019233, 'Plaza'), ('Doulon - Bottière', 'Carrefour City', 47.239462, -1.505152, 'Supermarket'), ('Doulon - Bottière', 'Marché du Vieux Doulon', 47.23554915879471, -1.5035901576918016, 'Market'), ('Doulon - Bottière', 'La Maie Du Vx Doulon', 47.235522328251214, -1.5041217542027645, 'Bakery')]]
[[('Nantes Sud', 'Aux Secrets du bien-etre', 47.180714, -1.510984, 'Health & Beauty Service'), ('Nantes Sud', 'Station Bourdonnières ➍', 47.187363358416754, -1.5209382601520582, 'Bus Stop'), ('Nantes Sud', 'Station Joliverie ➍', 47.189641580114206, -1.517180638489151, 'Bus Stop'), ('Nantes Sud', 'Station Chapeau-Verni ➍', 47.18924374359281, -1.5113440292726494, 'Bus Stop'), ('Nantes Sud', 'Quai Léon Sécher', 47.1821215466307, -1.5287761712527694, 'Harbor / Marina'), ('Nantes Sud', 'Station Porte de Vertou ➍', 47.1808750032925, -1.505151093006134, 'Bus Stop'), ('Nantes Sud', 'UrbanSoccer', 47.18961050810647, -1.5080350633998476, 'Soccer Field'), ('Nantes Sud', 'Station Mauvoisins ➍', 47.19014992434269, -1.525549292564392, 'Bus Stop')]]

In [84]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
print(nearby_venues)

  Neighborhood                      Venue  Venue Latitude  Venue Longitude  \
0   Nantes Sud   Aux Secrets du bien-etre       47.180714        -1.510984   
1   Nantes Sud    Station Bourdonnières ➍       47.187363        -1.520938   
2   Nantes Sud        Station Joliverie ➍       47.189642        -1.517181   
3   Nantes Sud    Station Chapeau-Verni ➍       47.189244        -1.511344   
4   Nantes Sud           Quai Léon Sécher       47.182122        -1.528776   
5   Nantes Sud  Station Porte de Vertou ➍       47.180875        -1.505151   
6   Nantes Sud                UrbanSoccer       47.189611        -1.508035   
7   Nantes Sud       Station Mauvoisins ➍       47.190150        -1.525549   

            Venue Category  
0  Health & Beauty Service  
1                 Bus Stop  
2                 Bus Stop  
3                 Bus Stop  
4          Harbor / Marina  
5                 Bus Stop  
6             Soccer Field  
7                 Bus Stop  


In [23]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
 
    categoryId = '4f4533804b9074f6e4fb0105'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    ES_results = requests.get(url).json()["response"]["venues"]
    ES_list= []
    for data in zip(ES_results):
        for d in data:
            E_name = d['name']
            E_lat = d['location']['lat']
            E_lng = d['location']['lng']
        ES_list.append([name, E_name, E_lat, E_lng])
    print(ES_list)

[['Centre-ville', 'Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886], ['Centre-ville', 'Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Centre-ville', 'Ecole Elementaire Ledru Rollin', 47.192533880262346, -1.5344116997318822], ['Centre-ville', 'Ecole Saint Anne', 47.20284502383568, -1.5798874929762246], ['Centre-ville', 'École Louise Michel', 47.205307, -1.53103], ['Centre-ville', 'École Bocage', 47.215716, -1.57043], ['Centre-ville', 'Ecole Elementaire Saint Nicolas', 47.21620515582818, -1.5628423242255631]]
[['Bellevue - Chantenay - Sainte-Anne', 'Ecole Primaire Du Fougan De Mer', 47.182608, -1.579806], ['Bellevue - Chantenay - Sainte-Anne', 'École primaire publique Les Reformes - Nantes', 47.20241173928157, -1.6032797440991424], ['Bellevue - Chantenay - Sainte-Anne', 'Ecole primaire Alphonse Braud', 47.20095406159978, -1.5843266422146052]]
[['Dervallières - Zola', 'Ecole Primaire Ampere', 47.2117805480957, -1.5846810340881348], ['Dervallières - Z

In [24]:
ES_lists = [['Centre-ville', 'Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886], ['Centre-ville', 'Ecole Saint Anne', 47.20284502383568, -1.5798874929762246], ['Centre-ville', 'Ecole Elementaire Ledru Rollin', 47.192533880262346, -1.5344116997318822], ['Centre-ville', 'Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Centre-ville', 'École Louise Michel', 47.205307, -1.53103], ['Centre-ville', 'École Bocage', 47.215716, -1.57043], ['Centre-ville', 'Ecole Elementaire Saint Nicolas', 47.21620515582818, -1.5628423242255631],
            ['Bellevue - Chantenay - Sainte-Anne', 'Ecole Primaire Du Fougan De Mer', 47.182608, -1.579806], ['Bellevue - Chantenay - Sainte-Anne', 'École primaire publique Les Reformes - Nantes', 47.20241173928157, -1.6032797440991424], ['Bellevue - Chantenay - Sainte-Anne', 'Ecole primaire Alphonse Braud', 47.20095406159978, -1.5843266422146052],
            ['Dervallières - Zola', 'Ecole Primaire Ampere', 47.2117805480957, -1.5846810340881348], ['Dervallières - Zola', 'École Du Soleil Levant', 47.219856, -1.610807], ['Dervallières - Zola', 'École maternelle Jacques Prévert', 47.2310637810822, -1.5771584660870175], ['Dervallières - Zola', 'École Bocage', 47.215716, -1.57043], ['Dervallières - Zola', 'École La Sensive', 47.211663891806744, -1.6080501009238382], ['Dervallières - Zola', 'Ecole Saint Anne', 47.20284502383568, -1.5798874929762246], ['Dervallières - Zola', 'École Villa Maria', 47.230804443359375, -1.5625208616256714], ['Dervallières - Zola', 'Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Dervallières - Zola', 'Ecole primaire Alain Fournier', 47.20373, -1.600433], ['Dervallières - Zola', 'École Élémentaire Contrie', 47.21561813354492, -1.600825548171997], ['Dervallières - Zola', 'Ecole élémentaire Mutualité', 47.2055, -1.589398], ['Dervallières - Zola', 'Ecole primaire Alphonse Braud', 47.20095406159978, -1.5843266422146052], ['Dervallières - Zola', 'Ecole Elementaire Saint Nicolas', 47.21620515582818, -1.5628423242255631],
            ['Hauts-Pavés - Saint-Félix', 'Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886], ['Hauts-Pavés - Saint-Félix', 'Ecole Primaire Fellonneau', 47.23161388241496, -1.5542852105734977], ['Hauts-Pavés - Saint-Félix', 'École Louise Michel', 47.205307, -1.53103], ['Hauts-Pavés - Saint-Félix', 'Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Hauts-Pavés - Saint-Félix', 'École Louis Guiotton', 47.224978586416455, -1.5274484876145378], ['Hauts-Pavés - Saint-Félix', 'École Villa Maria', 47.230804443359375, -1.5625208616256714], ['Hauts-Pavés - Saint-Félix', 'Ecole Saint Donatien', 47.23015972284917, -1.5426882631564025], ['Hauts-Pavés - Saint-Félix', 'Ecole catholique Saint Félix', 47.231649, -1.558304], ['Hauts-Pavés - Saint-Félix', 'École Bocage', 47.215716, -1.57043], ['Hauts-Pavés - Saint-Félix', 'Ecole Elementaire Saint Nicolas', 47.21620515582818, -1.5628423242255631],
            ['Malakoff - Saint-Donatien', 'École Louis Guiotton', 47.224978586416455, -1.5274484876145378], ['Malakoff - Saint-Donatien', 'Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Malakoff - Saint-Donatien', 'Ecole Publique Primaire Gay Lussac', 47.227520320903515, -1.5206311654520435], ['Malakoff - Saint-Donatien', 'Ecole Saint Donatien', 47.23015972284917, -1.5426882631564025], ['Malakoff - Saint-Donatien', 'Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886], ['Malakoff - Saint-Donatien', 'Ecole Primaire Fellonneau', 47.23161388241496, -1.5542852105734977], ['Malakoff - Saint-Donatien', 'École Louise Michel', 47.205307, -1.53103], ['Malakoff - Saint-Donatien', 'Ecole catholique Saint Félix', 47.231649, -1.558304],
            ['Île de Nantes', 'École Louise Michel', 47.205307, -1.53103], ['Île de Nantes', 'École maternelle publique Le Douet', 47.191412083954624, -1.5191573458834695], ['Île de Nantes', 'Ecole Elementaire Ledru Rollin', 47.192533880262346, -1.5344116997318822], ['Île de Nantes', 'École Sainte-Bernadette', 47.19280334684632, -1.5193163770973717], ['Île de Nantes', 'Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886],
            ['Breil - Barberie', 'Ecole Elementaire Saint Nicolas', 47.21620515582818, -1.5628423242255631], ['Breil - Barberie', 'École primaire publique Charles Lebourg', 47.23920408009994, -1.561497538937078], ['Breil - Barberie', 'La Perverie', 47.23962691927687, -1.561995238582545], ['Breil - Barberie', 'École maternelle Jacques Prévert', 47.2310637810822, -1.5771584660870175], ['Breil - Barberie', 'École Bocage', 47.215716, -1.57043], ['Breil - Barberie', 'Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886], ['Breil - Barberie', 'Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Breil - Barberie', 'Ecole primaire Alphonse Braud', 47.20095406159978, -1.5843266422146052], ['Breil - Barberie', 'Ecole élémentaire Mutualité', 47.2055, -1.589398], ['Breil - Barberie', 'École Élémentaire Contrie', 47.21561813354492, -1.600825548171997], ['Breil - Barberie', 'École Villa Maria', 47.230804443359375, -1.5625208616256714], ['Breil - Barberie', 'Ecole Saint Donatien', 47.23015972284917, -1.5426882631564025], ['Breil - Barberie', 'Ecole Primaire Fellonneau', 47.23161388241496, -1.5542852105734977], ['Breil - Barberie', 'Ecole Saint Anne', 47.20284502383568, -1.5798874929762246], ['Breil - Barberie', 'Ecole Primaire Ampere', 47.2117805480957, -1.5846810340881348], ['Breil - Barberie', 'Ecole catholique Saint Félix', 47.231649, -1.558304],
            ['Nantes Nord', 'École primaire publique Charles Lebourg', 47.23920408009994, -1.561497538937078], ['Nantes Nord', 'La Perverie', 47.23962691927687, -1.561995238582545], ['Nantes Nord', 'Ecole Don Bosco', 47.238283, -1.53342], ['Nantes Nord', 'Ecole catholique Saint Félix', 47.231649, -1.558304], ['Nantes Nord', 'Ecole Saint Donatien', 47.23015972284917, -1.5426882631564025], ['Nantes Nord', 'Ecole Primaire Fellonneau', 47.23161388241496, -1.5542852105734977],
            ['Nantes Erdre', 'Ecole Don Bosco', 47.238283, -1.53342], ['Nantes Erdre', 'École Louis Guiotton', 47.224978586416455, -1.5274484876145378], ['Nantes Erdre', 'Collège Public LA REINETIERE', 47.259533, -1.4916277], ['Nantes Erdre', 'Ecole Primaire Louis Pergaud', 47.26853382150955, -1.5184175814548964], ['Nantes Erdre', 'Ecole Publique Primaire Gay Lussac', 47.227520320903515, -1.5206311654520435],
            ['Doulon - Bottière', 'Ecole Publique Primaire Gay Lussac', 47.227520320903515, -1.5206311654520435], ['Doulon - Bottière', 'Ecole Saint Vincent', 47.250098, -1.488822], ['Doulon - Bottière', 'École Louis Guiotton', 47.224978586416455, -1.5274484876145378],
            ['Nantes Sud', 'École maternelle publique Le Douet', 47.191412083954624, -1.5191573458834695], ['Nantes Sud', 'Ecole Elementaire Ledru Rollin', 47.192533880262346, -1.5344116997318822], ['Nantes Sud', 'École Sainte-Bernadette', 47.19280334684632, -1.5193163770973717], ['Nantes Sud', 'École Henri Lesage', 47.18697825867775, -1.500406265258789]]

In [25]:
column_names = ['Neighbourhood', 'School Name', 'School Latitude', 'School Longitude']
ES_df = pd.DataFrame(columns = column_names)
    
for items in zip(ES_lists):
    for item in items:
        ES_df = ES_df.append({'Neighbourhood': item[0],'School Name': item[1],
                      'School Latitude': item[2],'School Longitude': item[3]}, ignore_index=True)

ES_df.shape

(80, 4)

In [62]:
ES_df['School Type']='Elementary'

In [63]:
ES_df.head()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Centre-ville,Ecole Emilie Pehant,47.211717,-1.549089,Elementary
1,Centre-ville,Ecole Saint Anne,47.202845,-1.579887,Elementary
2,Centre-ville,Ecole Elementaire Ledru Rollin,47.192534,-1.534412,Elementary
3,Centre-ville,Ecole Primaire Jean Jaures,47.219422,-1.559645,Elementary
4,Centre-ville,École Louise Michel,47.205307,-1.531030,Elementary


In [64]:
ES_df.tail()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
75,Doulon - Bottière,École Louis Guiotton,47.224979,-1.527448,Elementary
76,Nantes Sud,École maternelle publique Le Douet,47.191412,-1.519157,Elementary
77,Nantes Sud,Ecole Elementaire Ledru Rollin,47.192534,-1.534412,Elementary
78,Nantes Sud,École Sainte-Bernadette,47.192803,-1.519316,Elementary
79,Nantes Sud,École Henri Lesage,47.186978,-1.500406,Elementary


In [65]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
    print(name)
    
    categoryId = '4bf58dd8d48988d13d941735'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    HS_results = requests.get(url).json()['response']['venues']
    HS_list= []
    for data in zip(HS_results):
        for d in data:
            h_name = d['name']
            h_lat = d['location']['lat']
            h_lng = d['location']['lng']
        HS_list.append([name, h_name, h_lat, h_lng])
    print(HS_list)

Centre-ville
[['Centre-ville', 'Pôle des arts', 47.205251969088835, -1.5621783218762748], ['Centre-ville', 'Lycée La Baugerie', 47.206593, -1.520225], ['Centre-ville', 'Lycée Clémenceau', 47.219461221583146, -1.5448699352021085], ['Centre-ville', 'Lycée Jean Perrin', 47.181968828265965, -1.5613245964050293], ['Centre-ville', 'Lycée Nelson Mandela', 47.20839, -1.53202], ['Centre-ville', 'Lycée Saint Stanislas', 47.22273038539973, -1.555729114064831], ['Centre-ville', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Centre-ville', 'Esbanm', 47.216874, -1.554817], ['Centre-ville', 'Lycée Vial', 47.21829083967072, -1.56783642500402], ['Centre-ville', 'Salle 4071 - CCI Nantes', 47.206323, -1.575243], ['Centre-ville', 'Salle 3072 - CCI Nantes', 47.202059, -1.575276], ['Centre-ville', "lycée de l'île de Nantes", 47.20818847926514, -1.5335549236311614], ['Centre-ville', 'Wall Street Institute', 47.21768041632019, -1.5545538515370179], ['Centre-ville', "Lycée Gabriel Guist'Hau", 47.2163362624

In [66]:
HS_lists = [['Centre-ville', 'Pôle des arts', 47.205251969088835, -1.5621783218762748], ['Centre-ville', 'Lycée Jean Perrin', 47.181968828265965, -1.5613245964050293], ['Centre-ville', 'Lycée Nelson Mandela', 47.20839, -1.53202], ['Centre-ville', 'Lycée Clémenceau', 47.219461221583146, -1.5448699352021085], ['Centre-ville', 'Lycée Saint Stanislas', 47.22273038539973, -1.555729114064831], ['Centre-ville', "Lycée Gabriel Guist'Hau", 47.216336262403324, -1.568325161933899], ['Centre-ville', 'Esbanm', 47.216874, -1.554817], ['Centre-ville', 'Salle 4071 - CCI Nantes', 47.206323, -1.575243], ['Centre-ville', "lycée de l'île de Nantes", 47.20818847926514, -1.5335549236311614], ['Centre-ville', 'Wall Street Institute', 47.21768041632019, -1.5545538515370179], ['Centre-ville', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Centre-ville', 'Lycée La Baugerie', 47.206593, -1.520225], ['Centre-ville', 'Lycée Vial', 47.21829083967072, -1.56783642500402], ['Centre-ville', 'Salle 3072 - CCI Nantes', 47.202059, -1.575276], 
            ['Dervallières - Zola', 'Externat des Enfants Nantais', 47.215971257940815, -1.5727128852106873], ['Dervallières - Zola', "Collège ND de l'Abbaye", 47.209871, -1.592882], ['Dervallières - Zola', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Dervallières - Zola', 'Salle 3072 - CCI Nantes', 47.202059, -1.575276], ['Dervallières - Zola', "Lycée Gabriel Guist'Hau", 47.216336262403324, -1.568325161933899], ['Dervallières - Zola', 'Wall Street Institute', 47.21768041632019, -1.5545538515370179], ['Dervallières - Zola', 'Pôle des arts', 47.205251969088835, -1.5621783218762748], ['Dervallières - Zola', 'Lycée Saint Stanislas', 47.22273038539973, -1.555729114064831], ['Dervallières - Zola', 'ENSEC', 47.22377474023725, -1.564805303376603], ['Dervallières - Zola', 'Salle 4071 - CCI Nantes', 47.206323, -1.575243], ['Dervallières - Zola', 'Esbanm', 47.216874, -1.554817], ['Dervallières - Zola', 'Lycée Vial', 47.21829083967072, -1.56783642500402], 
            ['Hauts-Pavés - Saint-Félix', 'Lycée Clémenceau', 47.219461221583146, -1.5448699352021085], ['Hauts-Pavés - Saint-Félix', 'Externat des Enfants Nantais', 47.215971257940815, -1.5727128852106873], ['Hauts-Pavés - Saint-Félix', 'Lycée Nelson Mandela', 47.20839, -1.53202], ['Hauts-Pavés - Saint-Félix', 'Lycée Eugene Livet', 47.22605413309721, -1.5448382496045185], ['Hauts-Pavés - Saint-Félix', 'Lycée Saint Stanislas', 47.22273038539973, -1.555729114064831], ['Hauts-Pavés - Saint-Félix', 'Pôle des arts', 47.205251969088835, -1.5621783218762748], ['Hauts-Pavés - Saint-Félix', 'Wall Street Institute', 47.21768041632019, -1.5545538515370179], ['Hauts-Pavés - Saint-Félix', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Hauts-Pavés - Saint-Félix', 'Salle 4071 - CCI Nantes', 47.206323, -1.575243], ['Hauts-Pavés - Saint-Félix', 'ENSEC', 47.22377474023725, -1.564805303376603], ['Hauts-Pavés - Saint-Félix', 'Esbanm', 47.216874, -1.554817], ['Hauts-Pavés - Saint-Félix', 'Lycee Le Loquidy', 47.236376, -1.557094], ['Hauts-Pavés - Saint-Félix', 'Lycée Vial', 47.21829083967072, -1.56783642500402], ['Hauts-Pavés - Saint-Félix', 'Saint Jean Baptiste De La Salle', 47.23219183575799, -1.5600667846551706], ['Hauts-Pavés - Saint-Félix', "Lycée Gabriel Guist'Hau", 47.216336262403324, -1.568325161933899], ['Hauts-Pavés - Saint-Félix', "lycée de l'île de Nantes", 47.20818847926514, -1.5335549236311614],
            ['Malakoff - Saint-Donatien', 'Lycée Nelson Mandela', 47.20839, -1.53202], ['Malakoff - Saint-Donatien', 'Lycée Saint Stanislas', 47.22273038539973, -1.555729114064831], ['Malakoff - Saint-Donatien', 'Lycée Clémenceau', 47.219461221583146, -1.5448699352021085], ['Malakoff - Saint-Donatien', 'Lycée Eugene Livet', 47.22605413309721, -1.5448382496045185], ['Malakoff - Saint-Donatien', 'lycée horticole "le Grand Blottereau"', 47.225742, -1.51206], ['Malakoff - Saint-Donatien', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Malakoff - Saint-Donatien', "lycée de l'île de Nantes", 47.20818847926514, -1.5335549236311614], ['Malakoff - Saint-Donatien', 'Wall Street Institute', 47.21768041632019, -1.5545538515370179], ['Malakoff - Saint-Donatien', 'Lycée La Baugerie', 47.206593, -1.520225], ['Malakoff - Saint-Donatien', 'Esbanm', 47.216874, -1.554817], ['Malakoff - Saint-Donatien', 'Saint Jean Baptiste De La Salle', 47.23219183575799, -1.5600667846551706],
            ['Île de Nantes', 'Lycée Nelson Mandela', 47.20839, -1.53202], ['Île de Nantes', 'Lycée Polyvalent Les Bourdonnières', 47.18603508747084, -1.5218819611617012], ['Île de Nantes', 'Lycée La Joliverie', 47.19049988963339, -1.5165424346923828], ['Île de Nantes', 'Lycée Clémenceau', 47.219461221583146, -1.5448699352021085], ['Île de Nantes', 'Lycée La Baugerie', 47.206593, -1.520225], ['Île de Nantes', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Île de Nantes', "lycée de l'île de Nantes", 47.20818847926514, -1.5335549236311614],
            ['Breil - Barberie', "Lycée Gabriel Guist'Hau", 47.216336262403324, -1.568325161933899], ['Breil - Barberie', 'Externat des Enfants Nantais', 47.215971257940815, -1.5727128852106873], ['Breil - Barberie', "Collège ND de l'Abbaye", 47.209871, -1.592882], ['Breil - Barberie', 'Lycée Vial', 47.21829083967072, -1.56783642500402], ['Breil - Barberie', 'ENSEC', 47.21292055620176, -1.558673805058574], ['Breil - Barberie', 'Esbanm', 47.216874, -1.554817], ['Breil - Barberie', 'Saint Jean Baptiste De La Salle', 47.23219183575799, -1.5600667846551706], ['Breil - Barberie', 'Salle 3072 - CCI Nantes', 47.202059, -1.575276], ['Breil - Barberie', 'Wall Street Institute', 47.21768041632019, -1.5545538515370179], ['Breil - Barberie', 'La Perverie', 47.23962691927687, -1.561995238582545], ['Breil - Barberie', 'ENSEC', 47.22377474023725, -1.564805303376603], ['Breil - Barberie', 'Salle 4071 - CCI Nantes', 47.206323, -1.575243], ['Breil - Barberie', 'Lycée Eugene Livet', 47.22605413309721, -1.5448382496045185], ['Breil - Barberie', 'Pôle des arts', 47.205251969088835, -1.5621783218762748], ['Breil - Barberie', 'Lycée Clémenceau', 47.219461221583146, -1.5448699352021085], ['Breil - Barberie', 'Lycée Saint Stanislas', 47.22273038539973, -1.555729114064831], ['Breil - Barberie', 'Lycee Le Loquidy', 47.236376, -1.557094],
            ['Nantes Nord', 'La Perverie', 47.23962691927687, -1.561995238582545], ['Nantes Nord', 'Saint Jean Baptiste De La Salle', 47.23219183575799, -1.5600667846551706], ['Nantes Nord', 'demo nun evi', 47.25188446044922, -1.5719890594482422], ['Nantes Nord', 'Lycée Eugene Livet', 47.22605413309721, -1.5448382496045185], ['Nantes Nord', 'Lycee Le Loquidy', 47.236376, -1.557094],
            ['Nantes Erdre', 'Leonard De Vinci', 47.23810324103982, -1.5223291160509835],
            ['Doulon - Bottière', 'lycée horticole "le Grand Blottereau"', 47.225742, -1.51206], ['Doulon - Bottière', 'Leonard De Vinci', 47.23810324103982, -1.5223291160509835],
            ['Nantes Sud', 'Lycée Polyvalent Les Bourdonnières', 47.18603508747084, -1.5218819611617012], ['Nantes Sud', 'Lycée La Joliverie', 47.19049988963339, -1.5165424346923828]]

In [67]:
column_names = ['Neighbourhood', 'School Name', 'School Latitude', 'School Longitude']
HS_df = pd.DataFrame(columns = column_names)
    
for items in zip(HS_lists):
    for item in items:
        HS_df = HS_df.append({'Neighbourhood': item[0],'School Name': item[1],
                      'School Latitude': item[2],'School Longitude': item[3]}, ignore_index=True)

HS_df.shape

(87, 4)

In [68]:
HS_df['School Type']='High School'

In [69]:
HS_df.head()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Centre-ville,Pôle des arts,47.205252,-1.562178,High School
1,Centre-ville,Lycée Jean Perrin,47.181969,-1.561325,High School
2,Centre-ville,Lycée Nelson Mandela,47.208390,-1.532020,High School
3,Centre-ville,Lycée Clémenceau,47.219461,-1.544870,High School
4,Centre-ville,Lycée Saint Stanislas,47.222730,-1.555729,High School


In [70]:
HS_df.tail()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
82,Nantes Erdre,Leonard De Vinci,47.238103,-1.522329,High School
83,Doulon - Bottière,"lycée horticole ""le Grand Blottereau""",47.225742,-1.512060,High School
84,Doulon - Bottière,Leonard De Vinci,47.238103,-1.522329,High School
85,Nantes Sud,Lycée Polyvalent Les Bourdonnières,47.186035,-1.521882,High School
86,Nantes Sud,Lycée La Joliverie,47.190500,-1.516542,High School


In [71]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):

    categoryId = '4f4533814b9074f6e4fb0107'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    NS_results = requests.get(url).json()['response']['venues']
    NS_list= []
    for data in zip(NS_results):
        for d in data:
            n_name = d['name']
            n_lat = d['location']['lat']
            n_lng = d['location']['lng']
        NS_list.append([name, n_name, n_lat, n_lng])
    print(NS_list)

[['Centre-ville', 'Ecole Maternelle Sarah Bernard', 47.19281475347723, -1.5334441253006679], ['Centre-ville', 'Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Centre-ville', 'Ecole La Martelliere', 47.198434182417245, -1.5251898616871533], ['Centre-ville', 'Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['Centre-ville', 'École Publique maternelle Frédureau', 47.22096, -1.561459], ['Centre-ville', 'Ecole Maternelle Henri Bergson', 47.217549195870866, -1.5289408134500144], ['Centre-ville', "gab's school", 47.19194761342661, -1.5688559422646968]]
[['Bellevue - Chantenay - Sainte-Anne', 'Ecole Maternelle Du Fougan De Mer', 47.183549, -1.578055]]
[['Dervallières - Zola', 'Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Dervallières - Zola', 'Ecole Maternelle Dervalières Chézine', 47.22612380981445, -1.5972340106964111], ['Dervallières - Zola', 'Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['Dervallières - Zola', 'Ecole De La Cremetteri

In [72]:
NS_lists = [['Centre-ville', 'Ecole Maternelle Sarah Bernard', 47.19281475347723, -1.5334441253006679], ['Centre-ville', 'Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Centre-ville', 'Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['Centre-ville', 'École Publique maternelle Frédureau', 47.22096, -1.561459], ['Centre-ville', 'Ecole Maternelle Henri Bergson', 47.217549195870866, -1.5289408134500144], ['Centre-ville', "gab's school", 47.19194761342661, -1.5688559422646968], ['Centre-ville', 'Ecole La Martelliere', 47.198434182417245, -1.5251898616871533],
['Bellevue - Chantenay - Sainte-Anne', 'Ecole Maternelle Du Fougan De Mer', 47.183549, -1.578055],
['Dervallières - Zola', 'Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Dervallières - Zola', 'Ecole Maternelle Dervalières Chézine', 47.22612380981445, -1.5972340106964111], ['Dervallières - Zola', 'Ecole Georges Lafont', 47.237393325933816, -1.582577554034091], ['Dervallières - Zola', 'Ecole De La Cremetterie', 47.21575880762606, -1.6083585133926344], ['Dervallières - Zola', 'Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['Dervallières - Zola', 'École Publique maternelle Frédureau', 47.22096, -1.561459], ['Dervallières - Zola', 'Crèche Adelaïde', 47.225989, -1.560811],
['Hauts-Pavés - Saint-Félix', 'Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['Hauts-Pavés - Saint-Félix', 'Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Hauts-Pavés - Saint-Félix', 'École Publique maternelle Frédureau', 47.22096, -1.561459], ['Hauts-Pavés - Saint-Félix', 'Crèche Adelaïde', 47.225989, -1.560811], ['Hauts-Pavés - Saint-Félix', 'Multi-Accueil Michelet', 47.23180808485726, -1.5565053908187758], ['Hauts-Pavés - Saint-Félix', 'Crèche Bambou', 47.22756853492419, -1.5301671700231259], ['Hauts-Pavés - Saint-Félix', 'Ecole Maternelle Henri Bergson', 47.217549195870866, -1.5289408134500144], ['Hauts-Pavés - Saint-Félix', 'Ecole maternelle Port Boyer', 47.240474700927734, -1.5409560203552246],
['Malakoff - Saint-Donatien', 'Ecole Maternelle Henri Bergson', 47.217549195870866, -1.5289408134500144], ['Malakoff - Saint-Donatien', 'Ecole La Martelliere', 47.198434182417245, -1.5251898616871533], ['Malakoff - Saint-Donatien', 'Ecole Du Dr Teillais', 47.23435827286611, -1.5328169125379234], ['Malakoff - Saint-Donatien', 'Crèche Bambou', 47.22756853492419, -1.5301671700231259], ['Malakoff - Saint-Donatien', 'Multi-Accueil Michelet', 47.23180808485726, -1.5565053908187758],
['Île de Nantes', 'Ecole La Martelliere', 47.198434182417245, -1.5251898616871533], ['Île de Nantes', 'Ecole Maternelle Sarah Bernard', 47.19281475347723, -1.5334441253006679], ['Île de Nantes', 'Ecole Maternelle Henri Bergson', 47.217549195870866, -1.5289408134500144],
['Breil - Barberie', 'Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Breil - Barberie', 'École Publique maternelle Frédureau', 47.22096, -1.561459], ['Breil - Barberie', 'Crèche Adelaïde', 47.225989, -1.560811], ['Breil - Barberie', 'Multi-Accueil Michelet', 47.23180808485726, -1.5565053908187758], ['Breil - Barberie', 'Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['Breil - Barberie', 'La Perverie', 47.23962691927687, -1.561995238582545],
['Nantes Nord', 'Ecole maternelle Port Boyer', 47.240474700927734, -1.5409560203552246], ['Nantes Nord', 'La Perverie', 47.23962691927687, -1.561995238582545], ['Nantes Nord', 'Ecole Du Dr Teillais', 47.23435827286611, -1.5328169125379234], ['Nantes Nord', 'Multi-Accueil Michelet', 47.23180808485726, -1.5565053908187758],
['Nantes Erdre', 'Ecole Du Dr Teillais', 47.23435827286611, -1.5328169125379234], ['Nantes Sud', 'Ecole Sainte Famille', 47.18192244598738, -1.5108161057962333], ['Nantes Sud', 'Ecole Maternelle Sarah Bernard', 47.19281475347723, -1.5334441253006679], ['Nantes Sud', 'Ecole La Martelliere', 47.198434182417245, -1.5251898616871533]]

In [73]:
column_names = ['Neighbourhood', 'School Name', 'School Latitude', 'School Longitude']
NS_df = pd.DataFrame(columns = column_names)
for items in zip(NS_lists):
    for item in items:
        NS_df = NS_df.append({'Neighbourhood': item[0],'School Name': item[1],
                      'School Latitude': item[2],'School Longitude': item[3]}, ignore_index=True)

NS_df.shape

(45, 4)

In [74]:
NS_df['School Type']='Nursery'

In [75]:
NS_df.head()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Centre-ville,Ecole Maternelle Sarah Bernard,47.192815,-1.533444,Nursery
1,Centre-ville,Ecole Joseph Blanchard,47.212376,-1.564768,Nursery
2,Centre-ville,Multi-Accueil Nouvelle Vague,47.222302,-1.560678,Nursery
3,Centre-ville,École Publique maternelle Frédureau,47.220960,-1.561459,Nursery
4,Centre-ville,Ecole Maternelle Henri Bergson,47.217549,-1.528941,Nursery


In [76]:
NS_df.tail()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
40,Nantes Nord,Multi-Accueil Michelet,47.231808,-1.556505,Nursery
41,Nantes Erdre,Ecole Du Dr Teillais,47.234358,-1.532817,Nursery
42,Nantes Sud,Ecole Sainte Famille,47.181922,-1.510816,Nursery
43,Nantes Sud,Ecole Maternelle Sarah Bernard,47.192815,-1.533444,Nursery
44,Nantes Sud,Ecole La Martelliere,47.198434,-1.525190,Nursery


In [77]:
bilingual_schools = ['Sainte Madeleine','Ecole Saint Michel','Ecole Montessori de Nantes','Ecole elementaire Anatole de Monzie','College Briand','Lycee Mandela','International School of Nantes']

for school in bilingual_schools:
    address = '{}, Nantes, France'.format(school)
    geolocator = Nominatim(user_agent="nantes_explorer_1")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(school, latitude, longitude))

The geograpical coordinates of Sainte Madeleine are 47.2029752, -1.5737022.
The geograpical coordinates of Ecole Saint Michel are 47.21511845, -1.5702468749922927.
The geograpical coordinates of Ecole Montessori de Nantes are 47.2171231, -1.5495297.
The geograpical coordinates of Ecole elementaire Anatole de Monzie are 47.20285225, -1.5416172947901479.
The geograpical coordinates of College Briand are 47.24925265, -1.4947872170179148.
The geograpical coordinates of Lycee Mandela are 47.2083198, -1.533540256174522.
The geograpical coordinates of International School of Nantes are 47.210929, -1.613601.


In [146]:
prix = pd.read_csv('/Users/caroline/Desktop/BAROMETRE_PRIX_AVRIL_2021.csv')

In [126]:
prix.head()

,Quartier,Neuf,Ancien,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NANTES,4 670€,3 650 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Centre-ville,6 150€,4 380 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dervallières - Zola,5 230€,3 840 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hauts Pavés - Saint Félix,6 150€,4 070 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
prix_df = prix[['Quartier', 'Neuf', 'Ancien']]

In [148]:
prix_df.head(13)

,Quartier,Neuf,Ancien
0,NANTES,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Pavés - Saint Félix,6 150€,4 070 €
5,Saint Donatien - Malakoff,4 950€,3 770 €
6,Ile de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Quartiers Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


In [149]:
prix_df1 = pd.DataFrame(prix_df)

In [150]:
prix_df2 = prix_df1.rename(columns={'Quartier': 'Neighbourhood', 'Neuf': 'New Homes (per m2)', 'Ancien': 'Old Homes (per m2)'})
prix_df2.head()

,Neighbourhood,New Homes (per m2),Old Homes (per m2)
0,NANTES,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Pavés - Saint Félix,6 150€,4 070 €


In [151]:
prix_df2.loc[0,['Neighbourhood']] = 'Nantes'
prix_df2.loc[11,['Neighbourhood']] = 'Nantes Sud'
prix_df2.loc[4,['Neighbourhood']] = 'Hauts Paves - Saint Felix'
prix_df2.loc[6,['Neighbourhood']] = 'Ile de Nantes'
prix_df2.head(13)

,Neighbourhood,New Homes (per m2),Old Homes (per m2)
0,Nantes,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Paves - Saint Felix,6 150€,4 070 €
5,Saint Donatien - Malakoff,4 950€,3 770 €
6,Ile de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Quartiers Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


In [152]:
neighbourhoods_df.loc[-1] = ['Nantes', 0, 47.2186371, -1.5541362].copy()
neighbourhoods_df.index = neighbourhoods_df.index + 1
neighbourhoods_df = neighbourhoods_df.sort_index()
neighbourhoods_df.head(13)

,Neighbourhood,Population,Latitude,Longitude
0,Nantes,0,47.218637,-1.554136
1,Centre-ville,28 485,47.200142,-1.550053
2,Bellevue - Chantenay - Sainte-Anne,25 000,47.183478,-1.600042
3,Dervallières - Zola,35 000,47.216678,-1.583383
4,Hauts-Pavés - Saint-Félix,35 800,47.216794,-1.550108
5,Malakoff - Saint-Donatien,34 669,47.216736,-1.533361
6,Île de Nantes,15 818,47.200042,-1.533472
7,Breil - Barberie,24 418,47.216831,-1.566789
8,Nantes Nord,24 833,47.250031,-1.550128
9,Nantes Erdre,26 738,47.250092,-1.516767


In [153]:
neighbourhoods_df.loc[2,['Neighbourhood']] = 'Chantenay - Bellevue - Sainte-Anne'
neighbourhoods_df.loc[5,['Neighbourhood']] = 'Saint Donatien - Malakoff'
neighbourhoods_df.loc[8,['Neighbourhood']] = 'Quartiers Nord'
neighbourhoods_df.loc[4,['Neighbourhood']] = 'Hauts Paves - Saint Felix'
neighbourhoods_df.loc[6,['Neighbourhood']] = 'Ile de Nantes'
neighbourhoods_df.head(14)

,Neighbourhood,Population,Latitude,Longitude
0,Nantes,0,47.218637,-1.554136
1,Centre-ville,28 485,47.200142,-1.550053
2,Chantenay - Bellevue - Sainte-Anne,25 000,47.183478,-1.600042
3,Dervallières - Zola,35 000,47.216678,-1.583383
4,Hauts Paves - Saint Felix,35 800,47.216794,-1.550108
5,Saint Donatien - Malakoff,34 669,47.216736,-1.533361
6,Ile de Nantes,15 818,47.200042,-1.533472
7,Breil - Barberie,24 418,47.216831,-1.566789
8,Quartiers Nord,24 833,47.250031,-1.550128
9,Nantes Erdre,26 738,47.250092,-1.516767


In [154]:
neighbourhoods_df2 = pd.merge(neighbourhoods_df, prix_df2)
neighbourhoods_df2.head(13)

,Neighbourhood,Population,Latitude,Longitude,New Homes (per m2),Old Homes (per m2)
0,Nantes,0,47.218637,-1.554136,4 670€,3 650 €
1,Centre-ville,28 485,47.200142,-1.550053,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,25 000,47.183478,-1.600042,4 750€,3 450 €
3,Dervallières - Zola,35 000,47.216678,-1.583383,5 230€,3 840 €
4,Hauts Paves - Saint Felix,35 800,47.216794,-1.550108,6 150€,4 070 €
5,Saint Donatien - Malakoff,34 669,47.216736,-1.533361,4 950€,3 770 €
6,Ile de Nantes,15 818,47.200042,-1.533472,5 040€,3 390 €
7,Breil - Barberie,24 418,47.216831,-1.566789,4 780 €,3 230 €
8,Quartiers Nord,24 833,47.250031,-1.550128,4 720€,3 090 €
9,Nantes Erdre,26 738,47.250092,-1.516767,3 980 €,3 270 €
